In [1]:
import pathlib
from utils import get_filenames_of_path

In [2]:
root = pathlib.Path("/home/bdwumah74/Indiv_Proj")

inputs = get_filenames_of_path(root / 'Dataset')
inputs.sort()


for idx, path in enumerate(inputs):
    old_name = path.stem
    old_extension = path.suffix
    dir = path.parent
    new_name = str(idx).zfill(3) + old_extension
    path.rename(pathlib.Path(dir, new_name))

In [ ]:
import pathlib
from pytorch_faster_rcnn.visual import Annotator

from utils import get_filenames_of_path

dir = pathlib.Path("/home/bdwumah74/Indiv_Proj")
image_files = get_filenames_of_path(dir / 'Dataset')

annotator = Annotator(image_ids=image_files)
annotator.napari()

Cannot move to target thread (0x558858126d80)

